In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.offline as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import plotly.figure_factory as ff

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/glassdoor-analyze-gender-pay-gap/Glassdoor Gender Pay Gap.csv')

Vou separar nosso dataset para ficar mais fácil de entender nossos dados.

I'll split our dataset for stay more easy to understand

In [ ]:
femaleDs = df[df['Gender'] == 'Female'].loc[:,['BasePay','JobTitle', 'Education', 'Seniority']]
maleDs = df[df['Gender'] == 'Male'].loc[:,['BasePay','JobTitle', 'Education', 'Seniority']]

## Vamos caçar outliers  - Let's hunt outliers
A estrela da noite é o **boxplot**

The star of the night is the **boxplot**

O boxplot pode ser usado para analisar a dispersão dos dados, simetria, cauda e os outliers, 
ele representa nossos dados dividido por quartis, de um quartil para outro temos o intervalo interquartil, cada intervalo contém 25% dos dados. 

The boxplot can be used for analyse data distribution, simetry, tail and outliers. It represents our data split in quartiles, the range between two quartiles is called by interquartile range, each interquartile range contains 25% our data. 

Primeiro quartil representa os 25% primeiros valores, O segundo quartil (Q2) representa a mediana e   50% dos dados, e o Q3 representa 75% dos dados.

The first quartile(Q1) represents 25% of first values, the second quartile (Q2) represents the median and 50% of first values, the thirty quartile (Q3) represents 75% of first values.

Temos o ponto máximo que representa o nosso maior valor e o ponto minimo que é o menor valor da nossa distribuição. 

We have the min point it represents the min value of our distribution and max point it represents the max value.

Observe as linhas min, 25%, 50%, 75% e max.
Look the rows min, 25%, 50%, 75% e max.

In [ ]:
df.describe()

In [ ]:
boxFemale = go.Box(y = femaleDs.BasePay,
                name = 'Feminino',
                marker = {'color': '#e74c3c'})
boxMale = go.Box(y = maleDs.BasePay,
                name = 'Masculino',
                marker = {'color': '#00a000'})

data = [boxFemale, boxMale]

layout = go.Layout(title = 'Dispersão de salário por gênero',
                   titlefont = {'family': 'Arial',
                                'size': 22,
                                'color': '#7f7f7f'},
                   xaxis = {'title': 'Gênero'},
                   yaxis = {'title': 'Salário'},
                   paper_bgcolor = 'rgb(243, 243, 243)',
                   plot_bgcolor = 'rgb(243, 243, 243)')


fig = go.Figure(data=data, layout=layout)


## Vamos as visualizações - Let's go to Vis

Abaixo temos um boxplot, temos a visualização do dataset do sexo feminino e masculino

Under we have a boxplot, we can see two distributions, one for female (pink) and other for male (green).


In [ ]:
py.iplot(fig)

Só por esse gráfico já conseguimos notar algumas coisas a respeito dos nossos dados. Notem que o 2º e 3º quartil referente ao sexo masculino está entre 80k e 115k enquanto que para o feminino está entre 73k e 106k, e com isso a mediana é mais alta para o masculino.

If we look just this graphic, we can check somethings about our data. The Q2 and the Q3 of male boxplot is between 80k and 115k, while for female is between 73k and 106k, and for consequence the median it's smaller than male boxplot.

In [ ]:
def DistPlot(x, color):
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 6)
    sns.distplot(x, bins=100, color=color)

### Distribuição - Dist plot

Vamos plotar um gráfico de distribuição, nele conseguimos ver melhor a cauda, e a simetria, se é positiva ou negativa, se nossa cauda é longa ou não, etc.

Let's plot a distribution graphic, we can see the tail, if our tail its long or short, and symmetry, if it's positive or negative, etc. 

In [ ]:
DistPlot(maleDs['BasePay'], 'green')

Mais vis, porrqueeee ? Considero um ótimo jeito de conhecer nossos dados.
More vis, why ? Because vis its good way to know our data. 

In [ ]:
data = [maleDs['BasePay'],femaleDs['BasePay']]

group_labels = ['Masculino', 'Feminino']
colors = ['#00a000','#e74c3c']

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(data, group_labels, show_hist=False, colors=colors)

# Add title
fig.update_layout(title_text='Symmetry')
fig.show()

Com o gráfico de dispersão podemos ver onde tem mais/menos valores de uma forma simples.

This is a scatter plot, we can see focus of our data, points with more/less concentration of values.

In [ ]:
maleDs.sort_values(by=['BasePay'], inplace=True)

fig = px.scatter(maleDs, y=maleDs['BasePay'].sort_values(), 
                 color='BasePay',
                 labels={'y':'Salário (BasePay)'})
fig.show()

Podemos pegar um dado que foi classificado como outlier pelo índice. 

We can get a row was classified outlier by index.

In [ ]:
femaleDs.sort_values(by=['BasePay'], inplace=True)
maleDs.sort_values(by=['BasePay'], inplace=True)

maleDs['BasePay'].iloc[-2:] #get the last 2 rows

Fiquei com uma dúvida, a linha 927 é um outlier se considerarmos o macro escopo, mas se nós olharmos apenas para o cargo de gerente, será que também é um outlier ?

Well, I have a question, the row of index 927 its a outlier if we consider the macro scope, but if we consider just the job title Manager, let's see. 

In [ ]:
manager = maleDs.loc[(maleDs['JobTitle'] == 'Manager')]
manager

fig = px.box(manager, y="BasePay")
fig.show()

In his category, the outlier isn't outlier. So, have we drop any outlier ? Depends of context

## Dropping outliers

In [ ]:
femaleDs['BasePay'].iloc[-5:] # get the last 5 rows of dataframe female

In [ ]:
femaleDs = femaleDs.drop([357]) #droping outlier
maleDs = maleDs.drop([927, 353])  #droping outlier

## Conclusão - Conclusion

Sempre procure por outliers, entenda seus dados, entenda o objetivo da sua análise. 
Em alguns casos podemos tirar insights dos outliers. 
Outliers podem e vão atrapalhar suas análises e modelos. 

Always search for outliers, understand your data, understand the goal of your analysis.
In somecases, we can take insights of outliers. 
Outliers can and will mess up your analysis and models.